# Example of usage of Cross Validation while mixing models
Not well implemented yet


In [1]:
sc

# Import

In [56]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import itertools
from pyspark.mllib.recommendation import ALS
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 100


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [57]:
from pyspark.sql.functions import col

In [58]:
from cross_validation import KFoldIndexes,CrossValidation

In [59]:
from cross_validation_blending import CrossValidationBlending

In [60]:
from als import predictions_ALS
from means import user_mean,global_mean,movie_mean

In [61]:
import helpers

In [51]:
import random


# Dataframe creation

In [33]:
train = helpers.load_csv()
train.head()

,User,Movie,Rating
0,44,1,4
1,61,1,3
2,67,1,4
3,72,1,3
4,86,1,5


# Cross Validation Blending examples
Example on how CrossValidationBlending

In [62]:
blending_test=CrossValidationBlending(train,4)

In [63]:
blending_test.add_model(predictions_ALS,'als')
blending_test.add_model(predictions_ALS,'als2')

In [ ]:
blending_test.add_model(global_mean,'global_mean')

In [ ]:
blending_test.add_model(user_mean,'user_mean')

In [ ]:
blending_test.add_model(movie_mean,'movie_mean')

In [64]:
blending_test.add_params_for_model('als',{'spark_context':sc,'rank':8,'lambda_':0.081, 'iterations':24, 'nonnegative':True},compute_predictions=False)
blending_test.add_params_for_model('als2',{'spark_context':sc,'rank':8,'lambda_':0.081, 'iterations':24, 'nonnegative':True},compute_predictions=False)

In [ ]:
blending_test.add_params_for_model('user_mean',{})

In [ ]:
blending_test.add_params_for_model('movie_mean',{})

In [ ]:
blending_test.add_params_for_model('global_mean',{})

In [ ]:
blending_test.evaluate_blending({'als':0,'als2':0,'global_mean':0,'user_mean':0,'movie_mean':1})

In [25]:
blending_test.evaluate_blending({'als':0.5,'als2':0.5})

0.98919356521225099

In [65]:
blending_test.evaluate_blending_for_validation({'als':0.5,'als2':0.5},training,testing)

[ALS] applying


Py4JJavaError: An error occurred while calling o2127.predict.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 8505.0 failed 1 times, most recent failure: Lost task 2.0 in stage 8505.0 (TID 9712, localhost): java.net.SocketException: Socket is closed
	at java.net.Socket.getInputStream(Socket.java:876)
	at org.apache.spark.api.python.PythonRunner.compute(PythonRDD.scala:124)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:70)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:38)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:306)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:270)


# Test with a general prediction function
Example of usage of a function - for all function the usage should be the same

In [37]:
train_index=random.sample(range(1176952),1000000)
train_index.sort()
test_index=list(set(range(1176952))-set(train_index))
test_index.sort()

In [38]:
training=train.loc[train_index]
testing=train.loc[test_index]

In [ ]:
predictions_ALS(training,testing,sc,rank=8,lambda_=0.081, iterations=24, nonnegative=True).head()

# Usage of simple Cross Validation on a single model


# Models

In [ ]:
class ALSModel:
    
    def __init__(self):
        pass
    
    def fit(self,data,**arg):
        self.model = ALS.train(data, **arg)
    
    def predict(self,data):
        data_for_preditions=data.map(lambda x: (x[0], x[1]))
        self.predictions = self.model.predictAll(data_for_preditions).map(lambda r: ((r[0], r[1]), r[2]))
    
    def evaluate(self,data):
        rates_and_preds = data.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(self.predictions)
        error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean())
        return error

# Cross validation

In [ ]:
cv=CrossValidation(train,4,True,sc)

In [ ]:
ranks = [8]
lambdas = [0.08,0.081,0.082,0.083,0.084,0.085,0.086,0.087,0.088,0.089,0.09]
numIters = [24]
nbr_models = len(ranks)*len(lambdas)*len(numIters)

bestModel = None
bestValidationRmse = float("inf")
bestRank = 0
bestLambda = -1.0
bestNumIter = -1

In [ ]:
bestLambda

In [ ]:
i = 0
for rank, lmbda, numIter in itertools.product(ranks, lambdas, numIters):
    try:
        print(rank,lmbda,numIter)
        validationRmse = cv.evaluate(ALSModel(),rank=rank,lambda_=lmbda, iterations=numIter, nonnegative=True)
        validationRmse = np.mean(validationRmse)
        print("Model %i/%i: RMSE (validation) = %f" %(i+1, nbr_models, validationRmse))
        print("  Trained with rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))
        print("")
        if (validationRmse < bestValidationRmse):
#             bestModel = model
            bestValidationRmse = validationRmse
            bestRank = rank
            bestLambda = lmbda
            bestNumIter = numIter
    except:
        print("Model %i/%i failed!" %(i+1, nbr_models))
        print("  Parameters: rank = %d, lambda = %.1f, and numIter = %d." % (rank, lmbda, numIter))

    i += 1
    
# Evaluate the best model on the training set
print("The best model was trained with rank = %d and lambda = %.1f, " % (bestRank, bestLambda) \
  + "and numIter = %d, and its RMSE on the training set is %f" % (bestNumIter, bestValidationRmse))

# # Evaluate the best model on the test set
# testRmse = computeRMSE(bestModel, test_for_predict_RDD, test_RDD)
# print("RMSE on the test set: %f"%(testRmse))


# Prepare data for submission

In [ ]:
test = pd.read_csv('../data/sampleSubmission.csv')
test.head()

In [ ]:
# Prepare test for RDD
test_prep = test
test_prep['UserID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[0][1:]))
test_prep['MovieID'] = test_prep['Id'].apply(lambda x: int(x.split('_')[1][1:]))
test_prep['Rating'] = test_prep['Prediction']
test_prep = test_prep.drop(['Prediction', 'Id'], axis=1)
test_prep.head()

In [ ]:
# First, we transform it using sqlContect
test_sql = sqlContext.createDataFrame(test_prep)
test_rdd = test_sql.rdd
test_rdd.take(3)

In [ ]:
bestModel=ALSModel()
bestModel.fit(train_rdd,rank=8,lambda_=0.081, iterations=24, nonnegative=True)

In [ ]:
bestModel.predict(test_rdd)
predictions=bestModel.predictions
# predictions = bestModel.predictAll(test_RDD_Kaggle).map(lambda r: ((r[0], r[1]), r[2]))

In [ ]:
predictions.take(3)

In [ ]:
pred_df = predictions.toDF().toPandas()

In [ ]:
pred_df['UserID'] = pred_df['_1'].apply(lambda x: x['_1'])
pred_df['MovieID'] = pred_df['_1'].apply(lambda x: x['_2'])
pred_df['Rating'] = pred_df['_2']
pred_df = pred_df.drop(['_1', '_2'], axis=1)
pred_df.head()

In [ ]:
pred_df = pred_df.sort_values(by=['MovieID', 'UserID'])
pred_df.head()

In [ ]:
pred_df.index = range(len(pred_df))

In [ ]:
test['Prediction'] = pred_df['Rating']


In [ ]:
test.head()

In [ ]:
test = test.drop(['UserID', 'MovieID', 'Rating'], axis=1)


In [ ]:
test.to_csv('pred_pyspark_als.csv', index=False)